# RandomForestClassifier

In [8]:
import sys
sys.path.append("C:\\Users\\guilh\\OneDrive\\Documentos\\GitHub\\sistemasinteligentes")
import numpy as np
import pandas as pd
from src.si.data.dataset import Dataset
from src.io.csv_file import read_csv, write_csv 
from src.io.data_file import *
from src.si.model_selection.split import train_test_split
from src.si.models.random_forest_classifier import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier as SklearnRandomForestClassifier
from src.si.metrics.accuracy import accuracy
from src.si.models.knn_classifier import KNNClassifier
from src.si.models.logistic_regression import LogisticRegression
from src.si.models.decision_tree_classifier import DecisionTreeClassifier
from src.si.ensemble.stacking_classifier import StackingClassifier
from src.si.model_selection.cross_validate import k_fold_cross_validation
from src.si.model_selection.grid_search import grid_search_cv
from src.si.model_selection.randomized_search_cv import randomized_search_cv

In [9]:
filename = r"C:\Users\guilh\OneDrive\Documentos\GitHub\sistemasinteligentes\datasets\iris\iris.csv"

data = read_csv(filename, sep=",",features=True,label=True)
train, test = train_test_split(data, test_size=0.33, random_state=42)
model = RandomForestClassifier(n_estimators=10,min_sample_split=3, max_depth=3, mode='gini')
model.fit(train)
print("made by me:",model.score(test))

#sklearn
model_comparation = SklearnRandomForestClassifier(n_estimators=10, min_samples_split=3, max_depth=3)
model_comparation.fit(train.X, train.y)
print("sklearn:",accuracy(test.y, model_comparation.predict(test.X)))

made by me: 0.9795918367346939
sklearn: 0.9795918367346939


# StackingClassifier ensemble

In [10]:
filename_breast = r"C:\Users\guilh\OneDrive\Documentos\GitHub\sistemasinteligentes\datasets\breast_bin\breast-bin.csv"
breast=read_csv(filename_breast, sep=",",features=True,label=True)
train_data, test_data = train_test_split(breast, test_size=0.20, random_state=42)

#knnregressor
knn = KNNClassifier(k=2)

#logistic regression
LG=LogisticRegression(l2_penalty=1, alpha=0.001, max_iter=1000)

#decisiontreee
DT=DecisionTreeClassifier(min_sample_split=5, max_depth=5, mode='gini')

#final model
final_modelo=KNNClassifier(k=2)
modelos=[knn,LG,DT]
exercise=StackingClassifier(modelos,final_modelo)
exercise.fit(train_data)
print(exercise.score(test_data))

0.9856115107913669


sklearn compartion

In [4]:
from sklearn.ensemble import StackingClassifier as StackingClassifier_sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [5]:
#knnregressor
knn = KNeighborsClassifier(n_neighbors=2)

#logistic regression
lr=LogisticRegression(penalty='l2', C=0.1, max_iter=1000)

#decisiontreee
dt=DecisionTreeClassifier(min_samples_split=5, max_depth=5, criterion='gini')

#final model
final_model=KNeighborsClassifier(n_neighbors=2)
models=[('knn',knn),('lr',lr),('dt',dt)]
exercise=StackingClassifier_sklearn(estimators=models,final_estimator=final_model)
exercise.fit(train.X, train.y)
print(accuracy(test.y, exercise.predict(test.X)))

0.9795918367346939


# k_fold_cross_validation testing

In [11]:
filename_breast = r"C:\Users\guilh\OneDrive\Documentos\GitHub\sistemasinteligentes\datasets\breast_bin\breast-bin.csv"
breast=read_csv(filename_breast, sep=",",features=True,label=True)
model = LogisticRegression()
# cross validate the model
scores_ = k_fold_cross_validation(model, breast, cv=5, seed=1)
print(scores_)
print(f'Mean score: {np.mean(scores_)} +/- {np.std(scores_)}')

[0.9424460431654677, 0.9712230215827338, 0.9928057553956835, 0.9640287769784173, 0.9712230215827338]
Mean score: 0.9683453237410072 +/- 0.01615103907959975


# grid search

In [ ]:
filename_breast = r"C:\Users\guilh\OneDrive\Documentos\GitHub\sistemasinteligentes\datasets\breast_bin\breast-bin.csv"
breast=read_csv(filename_breast, sep=",",features=True,label=True)
# initialize the Logistic Regression model
knn = LogisticRegression()

# parameter grid
parameter_grid_ = {
    'l2_penalty': (1, 10),
    'alpha': (0.001, 0.0001),
    'max_iter': (1000, 2000)
}

# cross validate the model
results_ = grid_search_cv(knn,
                            breast,
                            hyperparameter_grid=parameter_grid_,
                            cv=3)

# print the results
print(results_)

# get the best hyperparameters
best_hyperparameters = results_['best_hyperparameters']
print(f"Best hyperparameters: {best_hyperparameters}")

# get the best score
best_score = results_['best_score']
print(f"Best score: {best_score}")

{'scores': [0.9669540229885057, 0.9683908045977012, 0.9669540229885057, 0.9669540229885057, 0.9669540229885057, 0.9669540229885057, 0.9655172413793104, 0.9669540229885057], 'hyperparameters': [{'l2_penalty': 1, 'alpha': 0.001, 'max_iter': 1000}, {'l2_penalty': 1, 'alpha': 0.001, 'max_iter': 2000}, {'l2_penalty': 1, 'alpha': 0.0001, 'max_iter': 1000}, {'l2_penalty': 1, 'alpha': 0.0001, 'max_iter': 2000}, {'l2_penalty': 10, 'alpha': 0.001, 'max_iter': 1000}, {'l2_penalty': 10, 'alpha': 0.001, 'max_iter': 2000}, {'l2_penalty': 10, 'alpha': 0.0001, 'max_iter': 1000}, {'l2_penalty': 10, 'alpha': 0.0001, 'max_iter': 2000}], 'best_hyperparameters': {'l2_penalty': 1, 'alpha': 0.001, 'max_iter': 2000}, 'best_score': 0.9683908045977012}
Best hyperparameters: {'l2_penalty': 1, 'alpha': 0.001, 'max_iter': 2000}
Best score: 0.9683908045977012


# Randomized_search_cv

In [13]:
filename_breast = r"C:\Users\guilh\OneDrive\Documentos\GitHub\sistemasinteligentes\datasets\breast_bin\breast-bin.csv"
breast=read_csv(filename_breast, sep=",",features=True,label=True)
model = LogisticRegression()
parameter_grid_ = {
        'l2_penalty':np.linspace(1, 10, 10),
        'alpha':  np.linspace(0.001, 0.0001, 100),
        'max_iter': np.linspace(1000, 2000, 200)
    }
results_ = randomized_search_cv(model,
                          breast,
                          hyperparameter_grid=parameter_grid_,
                          cv=3,
                          n_iter=100)

#print(results_)
#scores
print("all",results_["scores"])
# get the best score
best_score = results_['best_score']
print(f"Best avg score: {best_score}")
print("best hiper:", results_["best_hyperparameters"])
#no exercicio da aula 7 do grid o numero maximo de combinações é 8 entao
#cheguei a fazer com o  n_iter=8  para verificar se dava o mesmo resultado que em grid search e deu, o que oprova estar bem implementado
#deu isto :{'l2_penalty': 1, 'alpha': 0.0001, 'max_iter': 1000}, 'best_score': 0.9683908045977011}


all [0.9655172413793104, 0.9669540229885057, 0.9669540229885057, 0.9669540229885057, 0.9669540229885057, 0.9669540229885057, 0.9669540229885057, 0.9683908045977011, 0.9669540229885056, 0.9683908045977011, 0.9669540229885056, 0.9669540229885057, 0.9669540229885057, 0.9669540229885057, 0.9669540229885056, 0.9669540229885056, 0.9655172413793104, 0.9669540229885057, 0.9669540229885057, 0.9669540229885057, 0.9669540229885056, 0.9669540229885056, 0.9669540229885056, 0.9669540229885057, 0.9669540229885057, 0.9669540229885057, 0.9669540229885056, 0.9669540229885057, 0.9669540229885057, 0.9669540229885056, 0.9669540229885057, 0.9669540229885056, 0.9669540229885057, 0.9669540229885057, 0.9655172413793104, 0.9669540229885056, 0.9669540229885057, 0.9669540229885057, 0.9669540229885057, 0.9669540229885057, 0.9655172413793104, 0.9669540229885057, 0.9669540229885057, 0.9669540229885057, 0.9669540229885057, 0.9655172413793104, 0.9669540229885057, 0.9683908045977012, 0.9669540229885056, 0.9669540229885